## Pre-processing and Training Data Development For Network Security Scanner

### Set Up

In [1]:
# Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import featuretools as ft
from sklearn.preprocessing import StandardScaler

### Load Data

In [2]:
#Import Data
df1 = pd.read_csv('../data/processed/CICData_sampled_1_processed.csv')
df2 = pd.read_csv('../data/processed/CICData_sampled_2_processed.csv')
df3 = pd.read_csv('../data/processed/CICData_sampled_3_processed.csv')

In [3]:
# Check data imports
print(df1.head())
print(df2.head())
print(df3.head())

   Dst Port  Protocol  Flow Duration  Total Fwd Packet  Total Bwd packets  \
0     43461         6          23131                42                 43   
1     31367         6            126                 1                  2   
2     48964         6          25540                42                 43   
3        53        17           1070                 2                  2   
4      6881         6            313                 1                  2   

   Total Length of Fwd Packet  Total Length of Bwd Packet  \
0                       454.0                     25486.0   
1                         0.0                         0.0   
2                       470.0                     23342.0   
3                        74.0                       106.0   
4                         0.0                         0.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  ...  \
0                   66.0                    0.0               10.809524  ...   
1         

### Combine and Clean

In [4]:
# Compare columns and align to shared schema
cols1, cols2, cols3 = set(df1.columns), set(df2.columns), set(df3.columns)
common_cols = sorted(cols1 & cols2 & cols3)
print("Columns in df1/df2/df3:", len(cols1), len(cols2), len(cols3))
print("Common columns:", len(common_cols))
print("Only in df1:", sorted(cols1 - cols2 - cols3))
print("Only in df2:", sorted(cols2 - cols1 - cols3))
print("Only in df3:", sorted(cols3 - cols1 - cols2))

df1c = df1[common_cols].copy()
df2c = df2[common_cols].copy()
df3c = df3[common_cols].copy()

def clean_dataset(df_in: pd.DataFrame) -> pd.DataFrame:
    df_out = df_in.replace([np.inf, -np.inf], np.nan)
    dup_count = df_out.duplicated().sum()
    if dup_count:
        df_out = df_out.drop_duplicates()
        print("Dropped duplicates:", dup_count)

    all_null_cols = [c for c in df_out.columns if df_out[c].isna().all()]
    if all_null_cols:
        df_out = df_out.drop(columns=all_null_cols)
        print("Dropped all-null columns:", all_null_cols)

    num_cols = df_out.select_dtypes(include=[np.number]).columns
    cat_cols = [c for c in df_out.columns if c not in num_cols]
    df_out[num_cols] = df_out[num_cols].fillna(df_out[num_cols].median())
    for c in cat_cols:
        if df_out[c].isna().any():
            df_out[c] = df_out[c].fillna(df_out[c].mode().iloc[0])

    print("Remaining missing values:", int(df_out.isna().sum().sum()))
    return df_out

print("Cleaning df1...")
df1c = clean_dataset(df1c)
print("Cleaning df2...")
df2c = clean_dataset(df2c)
print("Cleaning df3...")
df3c = clean_dataset(df3c)

Columns in df1/df2/df3: 70 70 70
Common columns: 70
Only in df1: []
Only in df2: []
Only in df3: []
Cleaning df1...
Dropped duplicates: 13829
Remaining missing values: 0
Cleaning df2...
Dropped duplicates: 13807
Remaining missing values: 0
Cleaning df3...
Dropped duplicates: 14078
Remaining missing values: 0


In [5]:
df1c.head()

,ACK Flag Count,Active Max,Active Mean,Active Min,Active Std,Average Packet Size,Bwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Header Length,Bwd IAT Max,...,RST Flag Count,SYN Flag Count,Subflow Bwd Bytes,Subflow Bwd Packets,Subflow Fwd Bytes,Subflow Fwd Packets,Total Bwd packets,Total Fwd Packet,Total Length of Bwd Packet,Total Length of Fwd Packet
0,83,0.0,0.0,0.0,0.0,305.176471,1224798,25940,1384,3145.0,...,0,4,0,0,0,0,43,42,25486.0,454.0
1,3,0.0,0.0,0.0,0.0,0.000000,0,0,64,3.0,...,0,0,0,0,0,0,2,1,0.0,0.0
2,83,0.0,0.0,0.0,0.0,280.141176,993449,23812,1384,3963.0,...,0,4,0,0,0,0,43,42,23342.0,470.0
3,0,0.0,0.0,0.0,0.0,54.250000,168224,180,16,6.0,...,0,0,0,0,0,0,2,2,106.0,74.0
4,3,0.0,0.0,0.0,0.0,0.000000,0,0,64,3.0,...,0,0,0,0,0,0,2,1,0.0,0.0


### Split and Scale (Predefined: df1/train, df2/val, df3/holdout)

In [6]:
# Split features/label using predefined datasets
label_col = "Label"
missing_label = [name for name, df_check in [("df1", df1c), ("df2", df2c), ("df3", df3c)] if label_col not in df_check.columns]
if missing_label:
    raise ValueError(f"Label column '{label_col}' not found in: {missing_label}")

X_train = df1c.drop(columns=[label_col])
y_train = df1c[label_col]
X_val = df2c.drop(columns=[label_col])
y_val = df2c[label_col]
X_holdout = df3c.drop(columns=[label_col])
y_holdout = df3c[label_col]

print("Train/Val/Holdout shapes:", X_train.shape, X_val.shape, X_holdout.shape)

Train/Val/Holdout shapes: (86167, 69) (86189, 69) (85918, 69)


In [7]:
# Check the results
print("X_train sample:")
print(X_train.head())

X_train sample:
   ACK Flag Count  Active Max  Active Mean  Active Min  Active Std  \
0              83         0.0          0.0         0.0         0.0   
1               3         0.0          0.0         0.0         0.0   
2              83         0.0          0.0         0.0         0.0   
3               0         0.0          0.0         0.0         0.0   
4               3         0.0          0.0         0.0         0.0   

   Average Packet Size  Bwd Bulk Rate Avg  Bwd Bytes/Bulk Avg  \
0           305.176471            1224798               25940   
1             0.000000                  0                   0   
2           280.141176             993449               23812   
3            54.250000             168224                 180   
4             0.000000                  0                   0   

   Bwd Header Length  Bwd IAT Max  ...  RST Flag Count  SYN Flag Count  \
0               1384       3145.0  ...               0               4   
1                 64    

### Summary

Do to using tree-based modeling a scaler is not needed, also I have had my data separated earlier in my process. All thats left is to save the data and move to the next step.

### Save Outputs

In [8]:
# Save outputs
output_dir = "../data/processed"
X_train.to_csv(f"{output_dir}/X_train.csv", index=False)
X_val.to_csv(f"{output_dir}/X_val.csv", index=False)
X_holdout.to_csv(f"{output_dir}/X_holdout.csv", index=False)
y_train.to_csv(f"{output_dir}/y_train.csv", index=False)
y_val.to_csv(f"{output_dir}/y_val.csv", index=False)
y_holdout.to_csv(f"{output_dir}/y_holdout.csv", index=False)

print("Saved split datasets to", output_dir)

Saved split datasets to ../data/processed
